In [ ]:
import sys

sys.path.append("..")
from core.DataLoader import (
    DataPreprocessor,
    DataConfig,
    LoadConfig,
    get_load_config_from_yaml,
)
import numpy as np
from importlib import reload
import matplotlib.pyplot as plt
import yaml
import core.assignment_models as Models
import core
import keras


MODEL_DIR = f"models/assignment_transformer"
CONFIG_PATH = "../config/workspace_config.yaml"

import os

if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)


load_config = get_load_config_from_yaml(CONFIG_PATH)

DataProcessor = DataPreprocessor(load_config)


with open(CONFIG_PATH, "r") as file:
    data_configs = yaml.safe_load(file)
plt.rcParams.update({"font.size": 14})

data_config = DataProcessor.load_from_npz(
    data_configs["data_path"]["nominal"], max_events=4_000_000
)
X_train, y_train, _, _ = DataProcessor.split_even_odd()
del DataProcessor

2025-12-17 13:28:00.471859: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765974480.495299 3509014 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765974480.503067 3509014 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765974480.521980 3509014 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765974480.521999 3509014 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765974480.522002 3509014 computation_placer.cc:177] computation placer alr

In [ ]:
TransformerMatcher = Models.FeatureConcatTransformer(data_config, name=r"Transformer")

TransformerMatcher.build_model(
    hidden_dim=32,
    num_layers=4,
    num_heads=8,
    dropout_rate=0.1,
    compute_HLF=False,
)

TransformerMatcher.adapt_normalization_layers(X_train)

TransformerMatcher.compile_model(
    loss={
        "assignment": core.utils.AssignmentLoss(),
    },
    optimizer=keras.optimizers.AdamW(learning_rate=1e-4, weight_decay=1e-4),
    metrics={
        "assignment": [core.utils.AssignmentAccuracy()],
    },
)
TransformerMatcher.model.summary()

FeatureConcatTransformer is designed for classification tasks; regression targets will be ignored.


I0000 00:00:1765974580.977910 3509014 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15511 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:65:00.0, compute capability: 6.0


Building model without regression output.
Submodel inputs:  [<KerasTensor shape=(None, 6, 5), dtype=float32, sparse=False, ragged=False, name=jet_inputs>, <KerasTensor shape=(None, 2, 4), dtype=float32, sparse=False, ragged=False, name=lep_inputs>, <KerasTensor shape=(None, 1, 2), dtype=float32, sparse=False, ragged=False, name=met_inputs>]
Input tensor:  <KerasTensor shape=(None, 6, 5), dtype=float32, sparse=False, ragged=False, name=jet_inputs> jet_inputs
Input tensor:  <KerasTensor shape=(None, 2, 4), dtype=float32, sparse=False, ragged=False, name=lep_inputs> lep_inputs
Input tensor:  <KerasTensor shape=(None, 1, 2), dtype=float32, sparse=False, ragged=False, name=met_inputs> met_inputs


I0000 00:00:1765974583.084035 3511727 service.cc:152] XLA service 0x7f4964003e90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1765974583.084062 3511727 service.cc:160]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
2025-12-17 13:29:43.107906: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1765974583.173013 3511727 cuda_dnn.cc:529] Loaded cuDNN version 91500


139/943 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

...

Model: "FeatureConcatTransformerModel"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ jet_inputs          │ (None, 6, 5)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lep_inputs          │ (None, 2, 4)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_input_transform │ (None, 6, 5)      │          0 │ jet_inputs[0][0]  │
│ (InputPtEtaPhiELay… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_mask            │ (None, 6)         │          0 │ jet_inputs[0][0]  │
│ (GenerateMask)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lep_input_transform │ (None, 2, 4)      │          0 │ lep_inputs[0][0]  │
│ (InputPtEtaPhiELay… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ met_inputs          │ (None, 1, 2)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ compute_high_level… │ (None, 6, 2, 2)   │          0 │ jet_input_transf… │
│ (ComputeHighLevelF… │                   │            │ jet_mask[0][0],   │
│                     │                   │            │ lep_input_transf… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ met_input_transform │ (None, 1, 2)      │          0 │ met_inputs[0][0]  │
│ (InputMetPhiLayer)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ hlf_input_normaliz… │ (None, 6, 2, 2)   │          9 │ compute_high_lev… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ met_input_normaliz… │ (None, 1, 2)      │          5 │ met_input_transf… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lep_input_normaliz… │ (None, 2, 4)      │          9 │ lep_input_transf… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_input_normaliz… │ (None, 6, 5)      │         11 │ jet_input_transf… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 6, 4)      │          0 │ hlf_input_normal… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 2)         │          0 │ met_input_normal… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 8)         │          0 │ lep_input_normal… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 6, 9)      │          0 │ jet_input_normal… │
│ (Concatenate)       │                   │            │ reshape[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat_vector       │ (None, 6, 2)      │          0 │ flatten[0][0]     │
│ (RepeatVector)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 36,585 (142.93 KB)

 Trainable params: 36,551 (142.78 KB)

 Non-trainable params: 34 (152.00 B)

In [ ]:
TransformerMatcher.train_model(
    epochs=10,
    X_train=X_train,
    y_train=y_train,
    sample_weights=core.utils.compute_sample_weights(X_train, y_train),
    batch_size=1024,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss",
            factor=0.5,
            patience=5,
            min_lr=1e-6,
            verbose=1,
        ),
        keras.callbacks.EarlyStopping(
            monitor="val_loss",
            patience=15,
            restore_best_weights=True,
            verbose=1,
        ),
    ],
)

Epoch 1/10


2025-12-17 13:30:04.877453: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatTransformerModel_1/met_input_transform_1/assert_equal_1/Assert/Assert


1561/1562 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3550 - loss: 0.3177

2025-12-17 13:30:29.745642: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatTransformerModel_1/met_input_transform_1/assert_equal_1/Assert/Assert


1562/1562 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.3551 - loss: 0.3176

2025-12-17 13:30:41.005053: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatTransformerModel_1/met_input_transform_1/assert_equal_1/Assert/Assert
2025-12-17 13:30:43.404377: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatTransformerModel_1/met_input_transform_1/assert_equal_1/Assert/Assert


1562/1562 ━━━━━━━━━━━━━━━━━━━━ 51s 18ms/step - accuracy: 0.3551 - loss: 0.3176 - val_accuracy: 0.5818 - val_loss: 0.1012 - learning_rate: 1.0000e-04
Epoch 2/10
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 14s 9ms/step - accuracy: 0.5559 - loss: 0.1625 - val_accuracy: 0.6620 - val_loss: 0.0902 - learning_rate: 1.0000e-04
Epoch 3/10
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 14s 9ms/step - accuracy: 0.6267 - loss: 0.1050 - val_accuracy: 0.6914 - val_loss: 0.0859 - learning_rate: 1.0000e-04
Epoch 4/10
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 14s 9ms/step - accuracy: 0.6585 - loss: 0.0931 - val_accuracy: 0.7069 - val_loss: 0.0835 - learning_rate: 1.0000e-04
Epoch 5/10
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 14s 9ms/step - accuracy: 0.6754 - loss: 0.0894 - val_accuracy: 0.7155 - val_loss: 0.0815 - learning_rate: 1.0000e-04
Epoch 6/10
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 14s 9ms/step - accuracy: 0.6856 - loss: 0.0871 - val_accuracy: 0.7226 - val_loss: 0.0796 - learning_rate: 1.0000e-04
Epoch 7/10
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 14s 9ms/step -

In [ ]:
TransformerMatcher.save_model(f"{MODEL_DIR}/model.keras")

Model saved to models/assignment_transformer_HLF/model.keras
